In [ ]:
import pandas as pd
import math
import matplotlib.pyplot as plt
import numpy as np
import os

from scipy.optimize import curve_fit

from functionsFIMS import getAnalysisNumbers, plotPolyExamples
from runDataClass import runData

In [ ]:
if __name__ == '__main__':
    plotPolyExamples()

In [ ]:
if __name__ == '__main__':
    runNos = getAnalysisNumbers()
    simData = runData(runNos[-1])
    treenames = simData.getTreeNames()
    print(treenames)
    for name in treenames:
        simData.printColumns(name)

In [ ]:
if __name__ == '__main__':
    simData.plot2DFieldLines('Cathode')